# 200 - Datasets generation for nested-NER

Outputs:
Train, dev & test datasets for multilayers NER experiment 1 (with ref dataset) and experiment 2 (with pero OCR dataset)

<b>Experiment 1 : Groundtruth dataset</b>
* `m2-experiment_1_prepared_dataset_ref_io_camembert_ner`
* `m2-experiment_1_prepared_dataset_ref_io_pretrained_camembert_ner`
* `m2-experiment_1_prepared_dataset_ref_iob2_camembert_ner`
* `m2-experiment_1_prepared_dataset_ref_iob2_pretrained_camembert_ner`

<b>Experiment 2 : Pero OCR dataset</b>
* `m2-experiment_2_prepared_dataset_pero_ocr_io_camembert_ner`
* `m2-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner`
* `m2-experiment_2_prepared_dataset_pero_ocr_iob2_camembert_ner`
* `m2-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner`

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

1


In [2]:
import torch

torch.cuda.empty_cache()
torch.cuda.is_available()

True

## Initialization

In [3]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_2')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_2


## Constants

In [4]:
# GLOBAL CONSTANTS
import config
import os
from pathlib import Path

config.SPLIT_SEED = 42 # Random seed used in train/dev/test. Do not change it if you want to recreate the paper results.
config.DEBUG = False # If true, text versions of the spacy datasets will be saved along with the .spacy files.

MODEL_NAME = "pretrained_camembert_ner"
#camembert_ner OR
#pretrained_camembert_ner

In [5]:
# Save on disk
from camembert_utils.util_IO import _convert_tokenizer
print("Tokenizer called in util_IO.py")
_convert_tokenizer.name_or_path

2023-01-20 11:35:55.282684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 11:36:03.740290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-20 11:36:03.740770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-20 11:36:03.740796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Tokenizer called in util_IO.py


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'HueyNemud/das22-10-camembert_pretrained'

In [6]:
# Save on disk
from camembert_utils.util_IOB2 import _convert_tokenizer
print("Tokenizer called in util_IOB2.py")
_convert_tokenizer.name_or_path

Tokenizer called in util_IOB2.py


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'HueyNemud/das22-10-camembert_pretrained'

# 01. Experiment #1 : Reference dataset with joint-labels

In [7]:
import numpy as np
import pandas as pd
import csv
GOLD_REF = DATASETS / "41-ner_ref_from_pero/gold.csv"
assert GOLD_REF.exists()

#gold_reference = pd.read_csv(GOLD_REF, header=None, names=["ner_xml","book"],skipinitialspace='True')
with open(GOLD_REF,'r',encoding='utf8') as f:
    lines = f.readlines()
    res = []
    for line in lines:
        l = line.split('", "')
        res.append([l[0][1:],l[1][:-2]])
gold_reference = pd.DataFrame(res,columns=["ner_xml","book"])

In [8]:
#TITRE-H and TITRE-P labels to transformers NER Pipeline
for i in range(len(gold_reference)):
    if '<TITRE-H>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-H','TITREH')
    if '<TITRE-P>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-P','TITREP')
gold_reference

,ner_xml,book
0,"<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...",Bottin1_1820
1,"<PER>Dufay</PER>, <ACT>essayeur du commerce</A...",Bottin1_1820
2,"<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...",Bottin1_1820
3,"<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...",Bottin1_1820
4,"<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <SPAT...",Bottin1_1820
...,...,...
8440,"<PER>Lamarche</PER>, <ACT>géographe</ACT> , <S...",Notables_communaux_seine_1801
8441,"<PER>Lamarck</PER>, <ACT>membre de l'institut<...",Notables_communaux_seine_1801
8442,"<PER>Lamare</PER>, <ACT>notaire</ACT>, <SPAT><...",Notables_communaux_seine_1801
8443,"<PER>Lamarre</PER> , <ACT>carrier</ACT>, <SPAT...",Notables_communaux_seine_1801


In [9]:
gold_reference.groupby('book').count()

,ner_xml
book,
Bottin1_1820,231
Bottin1_1827,485
Bottin1_1837,706
Bottin3_1854a,844
Cambon_almgene_1841,647
Deflandre_1828,301
Deflandre_1829,377
DidotBottin_1860a,360
DidotBottin_1861a,388


In [10]:
from dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference.to_numpy())

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [11]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

In [12]:
from camembert_utils.tools import createStatsTab

createStatsTab(train,dev,test)

,Train,Dev,Test,All
Entity type,,,,
PER,6085,676,1685,8446
ACT,4568,519,1094,6181
DESC,262,30,79,371
TITRE,11,2,0,13
TITREH,225,33,43,301
TITREP,54,7,33,94
SPAT,6214,694,1747,8655
LOC,6871,762,1788,9421
FT,55,7,14,76


### Save test set for qualitative analysis

In [13]:
#Save test subset in csv format for qualitative analysis
with open("./ref_dataset_test_subset.csv",'w',encoding='utf8') as tfile:
    for line in test:
        tfile.write('"' + line[0] + '", "' + line[1] + '"\n')

## IO Labels

In [14]:
from camembert_utils.util_IO import save_dataset_io # Local import

# Save on disk
output_directory = OUT_BASE / f"m2-experiment_1_prepared_dataset_ref_io_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
print(output_directory)

for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset_io(output_directory, datasets, ["train","dev","test"], suffix=len(train))

/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_io_pretrained_camembert_ner


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

### IOB2 Labels

In [15]:
from camembert_utils.util_IOB2 import save_dataset_iob2 # Local import

# Save on disk
output_directory = OUT_BASE / f"m2-experiment_1_prepared_dataset_ref_iob2_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
print(output_directory)

for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset_iob2(output_directory, datasets, ["train","dev","test"], suffix=len(train))

/work/stual/res_ICDAR/method_2/m2-experiment_1_prepared_dataset_ref_iob2_pretrained_camembert_ner


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

# 0.2 # Experiment 2 : Pero OCR Dataset

In [16]:
import numpy as np
import pandas as pd
import csv

GOLD_REF = DATASETS / "31-ner_align_pero/gold.csv"
gold_reference = pd.read_csv(GOLD_REF, header=None, names=["ner_xml","book"],skipinitialspace='True')

In [17]:
#TITRE-H and TITRE-P labels to transformers NER Pipeline
for i in range(len(gold_reference)):
    if '<TITRE-H>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-H','TITREH')
    if '<TITRE-P>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-P','TITREP')
gold_reference

,ner_xml,book
0,"☞ T <PER>Dufant (Victor)</PER>, <ACT>libraire...",Bottin1_1820
1,"<PER>Dutay</PER>, <ACT>essayeur du commerce</A...",Bottin1_1820
2,"<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...",Bottin1_1820
3,"<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...",Bottin1_1820
4,"Y ☞ <PER>Dnten</PER>,<ACT>charentier</ACT>, <S...",Bottin1_1820
...,...,...
8440,"<PER>Lamarche</PER>, <ACT>geographe</ACT> , <S...",Notables_communaux_seine_1801
8441,"<PER>Lamarck</PER>, <ACT>membre de l&apos;inst...",Notables_communaux_seine_1801
8442,"<PER>Lamare</PER> , <ACT>notaire</ACT>, <SPAT>...",Notables_communaux_seine_1801
8443,"<PER>Lamarre</PER>, <ACT>carrier</ACT>, <SPAT>...",Notables_communaux_seine_1801


In [18]:
from dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference.to_numpy())

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [19]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

### Save test dataset

In [20]:
#Save test subset in csv format for qualitative analysis
with open("./pero_ocr_dataset_test_subset.csv",'w',encoding='utf8') as tfile:
    for line in test:
        tfile.write('"' + line[0] + '", "' + line[1] + '"\n')

### Entities count

In [21]:
from camembert_utils.tools import createStatsTab

createStatsTab(train,dev,test)

,Train,Dev,Test,All
Entity type,,,,
PER,6085,676,1685,8446
ACT,4568,519,1094,6181
DESC,262,30,79,371
TITRE,11,2,0,13
TITREH,225,33,43,301
TITREP,54,7,33,94
SPAT,6214,694,1747,8655
LOC,6871,762,1788,9421
FT,55,7,14,76


### IO Labels

In [22]:
from camembert_utils.util_IO import save_dataset_io # Local import

# Save on disk
output_directory = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_io_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
print(output_directory)

for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset_io(output_directory, datasets, ["train","dev","test"], suffix=len(train))

/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

### IOB2 Labels

In [ ]:
from camembert_utils.util_IOB2 import save_dataset_iob2 # Local import

# Save on disk
output_directory = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_iob2_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
print(output_directory)

for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset_iob2(output_directory, datasets, ["train","dev","test"], suffix=len(train))

/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]